In [199]:
from tkinter import *
from tkinter import ttk
import serial
import matplotlib.pyplot as plt
ser = serial.Serial('COM3',  baudrate=115200, timeout=1)

In [200]:
root = Tk()
root.geometry('820x820')
root['bg']="#333"
ca = Canvas(root, width=800, height=800)
ca.place(x=8,y=8)

timestr=StringVar()
lb=Label(root,textvariable=timestr)
lb.place(x=20,y=20,height=30)
style=ttk.Style(root)
style.configure(".",font="Arial 15")

INdata = 0
REDdata = [0]*200
IRdata = [0]*200

def tt():
    global INdata
    global REDdata
    global IRdata
    
    ca.delete("all")    
    
    for i in range(199):
        REDdata[i] = REDdata[i+1] 
        IRdata[i] = IRdata[i+1]         
    INdata = ser.readline() 
    REDdatastr,IRdatastr = INdata.decode().split()
    
    REDdata[199] = int(REDdatastr)
    IRdata[199] = int(IRdatastr)
    
    #print(REDdata[199],IRdata[199])
    
    #---------------------------------------------------------------------       
    #RED

    REDx=0
    REDpxmax=0
    REDpmax=0
    REDpxmin=0
    REDpmin=65535
    REDplv=250
    REDdr=1
    REDaxmax,REDaymax=[],[]
    REDaxmin,REDaymin=[],[]

    for REDp in REDdata:
        if REDdr==1:
            if REDp>REDpmax:
                REDpmax=REDp
                REDpxmax=REDx
            else:
                REDd=REDpmax-REDp
                if REDd > REDplv:
                    #print(REDx,REDpxmax,REDpmax)    
                    REDaxmax+=[REDpxmax]
                    REDaymax+=[REDpmax]
                    REDpmin=REDpmax
                    REDdr=0                      
        else:
            if REDp<REDpmin:
                REDpmin=REDp
                REDpxmin=REDx     
            else:
                REDd=REDp-REDpmin
                if REDd > REDplv:
                    #print(REDx,REDpxmin,REDpmin)
                    REDaxmin+=[REDpxmin]
                    REDaymin+=[REDpmin]                    
                    REDpmax=REDpmin
                    REDdr=1
        REDx+=1
    
    '''
    if len(REDaxmax) >=2 :
        REDaverage=0
        for i in range( REDaxmax[0] , REDaxmax[1] ):
            REDaverage += REDdata[i]
        REDaverage /= (REDaxmax[1] - REDaxmax[0])

        REDamplitude = REDaymax[0] - REDaymin[0]
    else:
        REDaverage=1
        REDamplitude=1
    '''
    #---------------------------------------------------------------------         
    #IR    
    
    IRx=0
    IRpxmax=0
    IRpmax=0
    IRpxmin=0
    IRpmin=131072
    IRplv=200
    IRdr=1
    IRaxmax,IRaymax=[],[]
    IRaxmin,IRaymin=[],[]

    for IRp in IRdata:
        if IRdr==1:
            if IRp>IRpmax:
                IRpmax=IRp
                IRpxmax=IRx
            else:
                IRd=IRpmax-IRp
                if IRd > IRplv:
                    #print(IRx,IRpxmax,IRpmax)    
                    IRaxmax+=[IRpxmax]
                    IRaymax+=[IRpmax]
                    IRpmin=IRpmax
                    IRdr=0                      
        else:
            if IRp<IRpmin:
                IRpmin=IRp
                IRpxmin=IRx     
            else:
                IRd=IRp-IRpmin
                if IRd > IRplv:
                    #print(REDx,REDpxmin,REDpmin)
                    IRaxmin+=[IRpxmin]
                    IRaymin+=[IRpmin]                    
                    IRpmax=IRpmin
                    IRdr=1
        IRx+=1
    '''    
    if len(IRaxmax) >= 2 :
        IRaverage=0
        for i in range( IRaxmax[0] , IRaxmax[1] ):
            IRaverage += IRdata[i]
        IRaverage /= (IRaxmax[1] - IRaxmax[0])

        IRamplitude = IRaymax[0] - IRaymin[0]    
    else:
        IRaverage=1
        IRamplitude=1
    '''
    
    t=0
    DrawIRdata=[0]*200
    OutIRdata=[0]*200
    filterDrawIRdata=[0]*200
    Drawaverage=0
    for i in range( len(IRaxmax)-1 ):
        for j in range( IRaxmax[t] , IRaxmax[t+1] ):
            #print( IRdata[j] )
            DrawIRdata[j] = IRdata[j]
            Drawaverage += IRdata[j]
            
        IRmaxd = IRaxmax[t+1] - IRaxmax[t]
        if IRmaxd==0:
            IRmaxd=1
        Drawaverage /= IRmaxd

        for j in range( IRaxmax[t] , IRaxmax[t+1] ):
            DrawIRdata[j] = 400 - (DrawIRdata[j] - Drawaverage)/2.5
            #print( DrawIRdata[j] )

        t=t+1
        Drawaverage=0
        
    for j in range(198):
        filterDrawIRdata[j] = (DrawIRdata[j] + DrawIRdata[j+1])/2
        if filterDrawIRdata[j]<1 or filterDrawIRdata[j]>799:
            filterDrawIRdata[j]=400
    
    #---------------------------------------------------------------------         
    #SpO2  
       
    REDamplitude=1 
    REDaverage=1
    IRamplitude=1 
    IRaverage=1
 
    if len(IRaxmax) >= 3 :
        IRaverage=0
        REDaverage=0
        for i in range( IRaxmax[0] , IRaxmax[1] ):
            REDaverage += REDdata[i]
            IRaverage += IRdata[i]         
        IRdd=IRaxmax[1]-IRaxmax[0]
        if IRdd<=0:
            IRdd=1
        REDaverage = REDaverage/IRdd - REDdata[ IRaxmin[0] ]  
        IRaverage = IRaverage/IRdd - IRdata[ IRaxmin[0] ]     
        
        REDamplitude = REDdata[ IRaxmax[0] ] - REDdata[ IRaxmin[0] ]       
        IRamplitude = IRdata[ IRaxmax[0] ] - IRdata[ IRaxmin[0] ]
    
        
        if REDamplitude <= 0 :
            REDamplitude=1
        if REDaverage <= 0 :
            REDaverage=1
        if IRamplitude <= 0 :
            IRamplitude=1 
        if IRamplitude <= 0 :
            IRamplitude=1            
    
        SpO2R = (REDamplitude / REDaverage) / (IRamplitude / IRaverage) 
        SpO2 = round( -45.06*SpO2R*SpO2R + 30.354*SpO2R + 94.845 )
        #SpO2 = round( -29.8*SpO2R + 116.42 )
        print(SpO2)  
         #print(REDamplitude,REDaverage,IRamplitude,IRaverage)
    '''
    print("RED:",REDaxmax,REDaxmin) 
    print("IR :",IRaxmax,IRaxmin) 
    print("-----------------------------------------") 
    '''
    #---------------------------------------------------------------------  
    #PPG
    
    dx=[]
    for i in range( len(IRaxmax)-1 ):
        dx += [ IRaxmax[i+1] - IRaxmax[i] ] 
    
    sumdx = sum(dx)
    if sumdx==0:
        sumdx=1      
    lendx = len(dx)
    if lendx==0:
        lendx=1    
      
    PPG = round(60*50 / (sumdx/lendx) )
    if PPG==3000:
        PPG=0  
    
    #print(PPG)  
    timestr.set(PPG)    
    
    #---------------------------------------------------------------------  
    
    xy=[]
    for i in range(len(filterDrawIRdata)):
        xy += ( 1150-i*8 , filterDrawIRdata[i] )           
    ca.create_line(xy, width=2, fill='blue')   
    
    #---------------------------------------------------------------------  
    
    ca.after(1, tt)

ca.after(1, tt)

mainloop()    


-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-16
-26
-20
-16
11
11
11
11
11
11
11
11
11
11
13
17
17
24
28
34
33
27
26
32
32
32
26
20
18
13
13
13
6
2
-2
-3
-12
-20
-23
-26
-14
16
-10
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-78
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
-257
47
49
49
49
49
49
49
49
49
49
49
49
49
49
50
50
50
51
52
53
56
59
65
69
71
-30
64
64
64
69
71
-628
95
95
95
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53
-53


In [170]:
#R=(501/31976.542857142857)/(835/104789.34285714285)
R=0.7
print( round( -45.06*R*R + 30.354*R + 94.845 ) )

94
